<a href="https://colab.research.google.com/github/mayarachew/IronyDetectionInTweets/blob/main/irony_detection_attention_bilstm_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!cp -r "/content/drive/My Drive/sentiment_analysis/irony_detection_EN_taskA" "irony_detection_EN"
#!cp -r "/content/drive/My Drive/sentiment_analysis/irony_detection_pt2" "irony_detection_pt2"

In [3]:
#!pip install tensorflow_text
#!pip install attention

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout, Flatten, Embedding, SimpleRNN,LSTM,Bidirectional
from keras.layers.embeddings import Embedding
from attention import Attention
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from collections import Counter

In [5]:
path_train = "irony_detection_EN/SemEval2018-T3-train-taskA_emoji.txt"
path_valid = "irony_detection_EN/SemEval2018-T3-valid-taskA_emoji.txt"
path_test = "irony_detection_EN/SemEval2018-T3_gold_test_taskA_emoji.txt"

In [6]:
df_train = pd.read_csv(path_train, sep='\t',encoding="UTF-8",low_memory = False)
df_valid = pd.read_csv(path_valid, sep='\t',encoding="UTF-8",low_memory = False)
df_test = pd.read_csv(path_test, sep='\t',encoding="UTF-8",low_memory = False)

df_train.head()

,Tweet index,Tweet text,Label
0,701,Ummm can I please have @KipMooreMusic for Chri...,0
1,2048,@TorteDeLini I'll check it out. :-),0
2,645,I hate when random people get in the hot tub w...,1
3,3233,Storm coming! #todayimloving http://t.co/hsNU...,1
4,3408,Today is already shaping up to be fabulous,1


In [7]:
df_test['Label'] = pd.Categorical(df_test['Label'])
df_test['Label'] = df_test['Label'].cat.codes

df_train['Label'] = pd.Categorical(df_train['Label'])
df_train['Label'] = df_train['Label'].cat.codes

df_valid['Label'] = pd.Categorical(df_valid['Label'])
df_valid['Label'] = df_valid['Label'].cat.codes

In [8]:
y_train = df_train['Label']
X_train = df_train['Tweet text']

y_test = df_test['Label']
X_test = df_test['Tweet text']

y_valid = df_valid['Label']
X_valid = df_valid['Tweet text']

In [9]:
df_train.dtypes


Tweet index     int64
Tweet text     object
Label            int8
dtype: object

In [10]:
num_classes = 2


results = Counter()
df_train['Tweet text'].str.lower().str.split().apply(results.update)
df_valid['Tweet text'].str.lower().str.split().apply(results.update)
df_test['Tweet text'].str.lower().str.split().apply(results.update)
total_vocab_size = len(results)

In [11]:
soma = 0

for item in df_train['Tweet text']:
  soma += len(item)
media = soma/len(df_train['Tweet text'])
print(f'Em média, cada tweets possui {media} palavras.')

Em média, cada tweets possui 87.95447101211923 palavras.


Hiperparametros

In [12]:
vocab_size = 1000 # make the top list of words (common words)
embedding_dim = 40
max_length = 40
bs = 32
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary

In [13]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

In [14]:
word_index = tokenizer.word_index
len(word_index)

11264

In [15]:
train_sequences = tokenizer.texts_to_sequences(X_train)

valid_sequences = tokenizer.texts_to_sequences(X_valid)

test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

X_valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

X_test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


Verificando se está tudo correto

In [16]:
print(X_train.iloc[3])
print(X_train_padded[3])

Storm coming! #todayimloving  http://t.co/hsNULfHz8f
[  1 310   1   8   6   7   1   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


In [17]:
X_train_padded.shape, X_valid_padded.shape

((3053, 40), (764, 40))

In [18]:
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
#y_test = to_categorical(y_test)

Segue o modelo LSTM

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=max_length, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Attention(max_length))
model.add(Dense(2))
model.add(Activation('sigmoid'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 40)            40000     
                                                                 
 bidirectional (Bidirectiona  (None, 40, 128)          53760     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 40, 128)           0         
                                                                 
 attention (Attention)       (None, 40)                26624     
                                                                 
 dense (Dense)               (None, 2)                 82        
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                        

In [20]:
from tensorflow.keras.optimizers import SGD,Adam
#sgd = SGD(learning_rate=0.05)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

opt = Adam(learning_rate=0.0001, decay=1e-6)
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

#model.compile(optimizer='adam',
#                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#                   metrics=['accuracy'])

In [21]:
model.fit(X_train_padded,y_train,validation_data=(X_valid_padded,y_valid),epochs=10,batch_size=bs)

Epoch 1/10
96/96 [==============================] - 22s 131ms/step - loss: 0.6932 - accuracy: 0.4903 - val_loss: 0.6938 - val_accuracy: 0.4581
Epoch 2/10
96/96 [==============================] - 7s 72ms/step - loss: 0.6923 - accuracy: 0.5313 - val_loss: 0.6937 - val_accuracy: 0.4581
Epoch 3/10
96/96 [==============================] - 5s 48ms/step - loss: 0.6906 - accuracy: 0.5329 - val_loss: 0.6940 - val_accuracy: 0.4699
Epoch 4/10
96/96 [==============================] - 5s 47ms/step - loss: 0.6751 - accuracy: 0.5683 - val_loss: 0.6933 - val_accuracy: 0.5654
Epoch 5/10
96/96 [==============================] - 5s 48ms/step - loss: 0.6481 - accuracy: 0.6191 - val_loss: 0.6726 - val_accuracy: 0.5942
Epoch 6/10
96/96 [==============================] - 5s 47ms/step - loss: 0.6190 - accuracy: 0.6620 - val_loss: 0.6673 - val_accuracy: 0.6060
Epoch 7/10
96/96 [==============================] - 5s 47ms/step - loss: 0.5730 - accuracy: 0.6987 - val_loss: 0.6833 - val_accuracy: 0.6126
Epoch 8/10


In [22]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plot_graphs(history, 'accuracy')
    plt.ylim(None, 1)
    #plt.xticks(range(0,20))
    plt.subplot(1, 2, 2)
    plot_graphs(history, 'loss')
    plt.ylim(0, None)
    #plt.xticks(range(0,20))

In [23]:
# import classification_report
from sklearn.metrics import classification_report

_, test_acc = model.evaluate(X_test_padded, to_categorical(y_test), verbose=0)
print(test_acc)

y_pred = model.predict(X_test_padded, batch_size=bs, verbose=2)

# get the class with highest probability for each sample
y_pred = np.argmax(y_pred, axis=-1)

# get the classification report
print(classification_report(y_test, y_pred))

0.6492347121238708
25/25 - 1s - 1s/epoch - 46ms/step
              precision    recall  f1-score   support

           0       0.71      0.70      0.71       473
           1       0.56      0.58      0.57       311

    accuracy                           0.65       784
   macro avg       0.64      0.64      0.64       784
weighted avg       0.65      0.65      0.65       784

